# Text als Netzwerk

Die computerlinguistische Aufbereitung von Texten kann als Ausgangspunkt für weitere Analysen dienen, die typische Muster und Inhalte in Texten sichtbar machen. Eine Grundidee ist dabei, dass Elemente sich nicht allein durch ihre individuellen Eigenschaften auszeichnen, sondern vor allem durch ihre Relationen zu anderen Elementen. In der Soziologie hat dies etwa Mustafa Emirbayer (1997) herausgestellt. Dies gilt aber nicht nur für soziale Akteure, sondern auch für Sprache selbst. So hat etwa bereits Ludwig Wittgenstein in seinen *Philosophischen Untersuchungen* (2008 [1953]) die Verwandtschaft von Begriffen als ein „kompliziertes Netz von Ähnlichkeiten“ (ebd., 57) beschrieben.

Diese abstrakten Ideen lassen sich mit den Methoden der Netzwerkanalyse operationalisieren. Ein Netzwerk ist dabei zunächst eine Möglichkeit, Beziehungen zwischen Elementen abzubilden. Die gängigen Attributdaten der Soziologie werden zumeist als Tabelle dargestellt, bei der jede Zeilen einen Fall und jede Spalte dessen Eigenschaften repräsentiert. Im Netzwerk kommt eine zweite Ebene hinzu: Die Beziehungen zwischen diesen Fällen, wobei die einzelnen Verbindungen wieder Eingenschaften haben können (etwa Art, Intensität, Dauer, etc.).

In der Sozialforschung werden in der Regel Netzwerke von Akteuren analysiert. Auch solche Netzwerke lassen sich prinzipiell aus Texten gewinnen (etwa Diesner und Carley 2005). Für die Analyse von Texten lassen sich aber auch inhaltliche (semantische) Aspekte als Netzwerk darstellen und analysieren.

In [1]:
import networkx as nx
import pandas as pd
from textblob_de import TextBlobDE as TextBlob
from itertools import combinations
from pprint import pprint

Da wir hierfür die Verfahren der computerlinguistischen Sprachverarbeitung nutzen können, definieren wir zunächst wieder eine Funktion zur Lemmatisierung und zur Part-of-Speech-Filterung, die die Funktionen von `TextBlob` nutzt.

Die Wörter werden dabei nach drei Kriterien gefiltert:

* POS-Tags (wobei nur die ersten beiden Buchstaben der Tags berücksichtigt werden, die den Oberkategorien entsprechen, z.B. 'NN' für Substantive),
* eine Stopwortliste,
* und ein regulärer Ausdruck, der dazu dient, Zahlen enthaltende Wörter (z.B. '1990er', '18jährige') zu entfernen.

In [2]:
import re
import codecs
with codecs.open('../Daten/stopwords.txt', 'r', 'utf-8') as stopwordfile:
    stopwords = [line.strip() for line in stopwordfile.readlines()]

def lemmatize_and_filter(blob, tags):
    lemmas = blob.words.lemmatize()
    tagged = blob.pos_tags
    result = []
    for lemma, (word, tag) in zip(lemmas, tagged):
        if tag[0:2] in tags and not word in stopwords and re.match(u'[^\W0-9]+', lemma, re.U):
            result.append(lemma)
    return result

Ein einfacher Ansatz, Relationen von Texteinheiten (Wörtern) zu bestimmen, ist die Kookkurrenz: Zwei Wörter, die in räumlicher Nähe zueinander stehen, haben eine erhöhte Wahrscheinlichkeit, auch inhaltlich aufeinander bezogen zu sein. Als räumliche Nähe kann z.B. ein Satz definiert werden. Es sind aber auch andere Ansätze möglich, etwa ein fester Wortabstand von etwa 5 Wörtern. Alle (gefilterten) Wörter im jeweiligen Fenster sollen also als vernetzt gelten. Dazu muss für jede Zweierkombination aller Wörter im Fenster eine Relation erzeugt werden.

In Python steht dafür die Funktion `combinations()` zur Verfügung:

In [3]:
[(a, b) for a, b in combinations('abc', 2)]

[('a', 'b'), ('a', 'c'), ('b', 'c')]

Die Python-Bibliothek [NetworkX](http://networkx.github.io/) stellt einfache Funktionen bereit, ein Netzwerk zu erzeugen. Sie ist dabei allerdings nicht besonders schnell. Für größere Projekte bieten sich andere Bibliotheken wie [igraph](http://igraph.org/python/) an.

Das Kernvorgehen ist dabei sehr einfach:

1. Das Corpus ist in einer CSV-Datei gespeichert, wobei die Spalte 'text' den eigentlichen Text enhält.
2. Für jeden Text wird ein `TextBlob` erzeugt, der linguistische Informationen ergänzt, so u.a. auch Satzgrenzen.
3. Für jeden Satz werden die zu berücksichtigenden Lemmata bestimmt. In diesem Fall werden nur Substantive und Adjektive einbezogen.
4. Es werden für jeden Satz alle möglichen Kombinationen dieser Lemmata ermittelt.
5. Falls eine bestimmte Relation schon im Netzwerk enthalten ist, wird ihr „Gewicht“ um 1 erhöht, andernfalls wird eine neue Relation mit dem Gewicht 1 hinzugefügt.

In [4]:
data = pd.read_csv('../Daten/reden.csv', parse_dates=['date'], encoding='utf-8')
graph = nx.Graph()
for text in data['text']:
    blob = TextBlob(text)
    for sentence in blob.sentences:
        lemmas = lemmatize_and_filter(sentence, ('NN', 'JJ'))
        for a, b in combinations(lemmas, 2):
            if a == b:
                continue
            try:
                graph[a][b]['weight'] += 1
            except KeyError:
                graph.add_edge(a, b, weight=1)

In [5]:
u'Nodes: {}, Edges: {}'.format(graph.number_of_nodes(), graph.number_of_edges())

u'Nodes: 46085, Edges: 952228'

Um diese Prozedur nicht für jede Analyse wiederholen zu müssen, kann das Ergebnis in einer Datei gespeichert werden.

In [6]:
nx.write_graphml(graph, '../Daten/network.graphml')

Dieses Netzwerk kann nun mit Mitteln der Netzwerkanalyse analysiert werden. Eine einfache Netzwerkstatistik ist der “degree”, die Anzahl der Verbindungen jedes Knotens. `graph.degree()` aus NetworkX gibt dabei in einem `dict` für jeden Knoten seinen degree aus. Die Python-Klasse `Counter` stellt für solche dictionaries einige Zusatzfunktionen bereit, etwa die Ausgabe der `n` Knoten mit dem höchsten degree.

In [7]:
from collections import Counter
nodes_by_degree = Counter(graph.degree())
tops = nodes_by_degree.most_common(20)
tops

[(u'Deutschland', 7821),
 (u'gross', 6254),
 (u'Menschen', 5721),
 (u'deutsch', 5360),
 (u'Jahren', 5282),
 (u'Herren', 5118),
 (u'Damen', 5087),
 (u'Deutsch', 4655),
 (u'Jahr', 4579),
 (u'Beispiel', 4578),
 (u'Herr', 4256),
 (u'Europa', 4239),
 (u'Welt', 3916),
 (u'lieb', 3705),
 (u'Zeit', 3594),
 (u'Jahre', 3507),
 (u'Frage', 3494),
 (u'Land', 3445),
 (u'europ\xe4isch', 3409),
 (u'international', 3396)]

Das Gesamtnetzwerk mit seinen 46.000 Knoten ist dabei nicht leicht zu analysieren und inhaltlich zu interpretieren. Es kann aber ein Ausgangspunkt für Detailanalysen sein. Ausgehend von der Annahme, dass sich die Bedeutung eines Wortes durch seinen Verwendungszusammenhang ergibt, lassen sich etwa Begriffsanalysen durchführen. Hierfür kann für ein Wort ein Teilnetzwerk erstellt werden, das nur die mit ihm direkt verbundene Wörter („Nachbarn“) und deren Verbindungen untereinander enthält.

Als Beispiel soll das Wort „Integration“ dienen.

In [8]:
subgraph = graph.subgraph(graph.neighbors(u'Integration'))
u'Nodes: {}, Edges: {}'.format(subgraph.number_of_nodes(), subgraph.number_of_edges())

u'Nodes: 861, Edges: 48052'

Da eher zufällige Verbindungen das Ergebnis verzerren und ja systematische Beziehungen analysiert werden sollen, können die Kanten im Netzwerk noch gefiltert werden. Hier werden alle Kanten, die ein Gewicht von 1 haben, gelöscht. Es können auch höhere Schwellenwerte festgelegt oder aber ausgefeiltere statistische Kriterien zugrundegelegt werden, die die Signifikanz von Verbindungen bestimmen.

In [9]:
lowedges = [edge for edge in subgraph.edges(data=True) if edge[2]['weight'] < 2]
len(lowedges)

24808

Durch das Entfernen von Kanten können im Netzwerk „Inseln“ entstehen, die nicht mehr mit dem Gesamtnetzwerk verbunden sind:

In [10]:
subgraph.remove_edges_from(lowedges)
nx.number_connected_components(subgraph)

122

Diese werden für die weitere Analyse ausgeschlossen.

*Hinweis:* Hier wird ein Kniff angewandt, der möglichst ressourcensparend arbeitet, aber nicht auf den ersten Blick eingängig ist. Daher hier eine kurze Erläuterung.

Die Funktion `connected_component_subgraphs` gibt für jede verbundene Komponente des Netzwerks ein Teilnetzwerk zurück. Da dies aber rechenaufwändig ist und potenziell viel Speicher benötigt, ist das Ergebnis der Funktion keine Liste aller Teilnetzwerke, sondern ein „Generator“. Ein Generator unterscheidet sich von einer Liste dadurch, dass seine Elemente erst „generiert“ werden, wenn auf sie zugegriffen wird. Dabei erlaubt er nur die Form `for x in y`, nicht aber `y[i]`. Ein Generator kann aber in eine Liste umgewandelt werden: `list(y)`.

Die Komponenten in `connected_component_subgraphs` sind nach ihrer Größe geordnet, wobei das größte Teilnetzwerk zuerst ausgegeben wird. Um also nur das größte Teilnetzwerk zu erhalten wäre grundsätzlich dieses Vorgehen möglich (und auch nachvollziehbarer):

```python
subgraphs = list(nx.connected_component_subgraphs(subgraph))
subgraph2 = subgraphs[0]
```

Dafür müssten aber zunächst alle Teilnetzwerke erzeugt und in der Liste `subgraphs` gespeichert werden, obwohl sie nicht benötigt werden. Daher kann hier ein anderer Weg gewählt werden: In der Schleife `for subgraph2 in nx_connected_component_subgraphs(subgraph):` wird im ersten Durchgang das erste Teilnetzwerk erzeugt und der Variable `subgraph2` zugewiesen. Da das alles ist, was wir benötigen, kann die Schleife direkt danach abgebrochen werden, ohne dass weitere Befehle ausgeführt werden.

In [11]:
for subgraph2 in nx.connected_component_subgraphs(subgraph):
    break
u'Nodes: {}, Edges: {}'.format(subgraph2.number_of_nodes(), subgraph2.number_of_edges())

u'Nodes: 739, Edges: 23243'

Für die Analyse des Begriffsnetzwerkes von „Integration“ sind diejenigen Bereiche von Interesse, die eine dichtere Vernetzung aufweisen. Diese Teilnetzwerke sind nicht „Komponenten“ im oben genannten Sinne, da sie durchaus Teil des Gesamtnetzwerkes sind. Sie zu identifizieren kann also nur näherungsweise geschehen, es gibt kein absolutes Ergebnis. Dementsprechend sind in den letzten Jahren zahlreiche Algorithmen zur sogenannten “community detection” entwickelt worden, die sich im Detail und in ihrer Effizienz unterscheiden. Ein etablierter Algorithmus für NetworkX ist in [diesem Modul](http://perso.crans.org/aynaud/communities/) verfügbar.

*Hinweis:* Die Funktion `best_partition()` gibt für jeden Knoten im Netzwerk die Nummer seiner Community zurück:

    {'node1': 0,
     'node2': 1,
     'node3': 0}

Diese Form ist aber nur bedingt geeignet, um die Communities weiter zu analysieren. Besser geeignet wäre eine Liste aller Knoten für jede Community:

    [['node1', 'node3'], ['node2']]

Daher wird zunächst eine Liste erzeugt, die für jeden möglichen Community-Wert eine leere Liste enthält. Im nächsten Schritt wird jeder Knoten der seiner Community entsprechenden Liste hinzugefügt. Dadurch wird die gewünschte Struktur hergestellt.

Im nächsten Schritt kann dann für jede Community ein Teilnetzwerk erzeugt werden. Der Degree der Knoten in den Teilnetzwerken gibt Aufschluss über die zentralen Begriffe dieser Teilnetzwerke.

In [13]:
import community
partition = community.best_partition(subgraph2)

communities = [[] for i in set(partition.values())]
for node, comm in partition.items():
    communities[comm].append(node)
    
community_graphs = [subgraph2.subgraph(nodes) for nodes in communities]
for i, graph in enumerate(community_graphs):
    print u'Community {}:'.format(i)
    pprint(Counter(graph.degree()).most_common(10))

Community 0:
[(u'Europa', 124),
 (u'europ\xe4isch', 116),
 (u'Europ\xe4isch', 107),
 (u'Welt', 106),
 (u'Beispiel', 99),
 (u'gemeinsam', 98),
 (u'Union', 97),
 (u'L\xe4nder', 96),
 (u'L\xe4ndern', 87),
 (u'Weg', 81)]
Community 1:
[(u'Menschen', 146),
 (u'Frage', 119),
 (u'Zukunft', 116),
 (u'Gesellschaft', 113),
 (u'Politik', 94),
 (u'Thema', 91),
 (u'Leben', 89),
 (u'Aufgabe', 88),
 (u'politisch', 88),
 (u'wichtig', 86)]
Community 2:
[(u'Deutschland', 147),
 (u'deutsch', 114),
 (u'gross', 106),
 (u'Jahren', 96),
 (u'Land', 90),
 (u'Jahre', 77),
 (u'Zeit', 77),
 (u'international', 77),
 (u'Geschichte', 76),
 (u'Seite', 66)]
Community 3:
[(u'Herr', 91),
 (u'lieb', 89),
 (u'Herren', 78),
 (u'Damen', 78),
 (u'Deutsch', 75),
 (u'Frau', 67),
 (u'herzlich', 57),
 (u'Professor', 50),
 (u'freue', 46),
 (u'Berlin', 43)]
Community 4:
[(u'Jahr', 24),
 (u'Bundesregierung', 22),
 (u'Euro', 20),
 (u'Rahmen', 17),
 (u'Bildung', 17),
 (u'Millionen', 15),
 (u'Bereich', 15),
 (u'kulturell', 15),
 (u'zus

Diese Ausgabe gibt einen ersten Anhaltspunkt für die Interpretation des Begriffs „Integration“:

1. Die erste Community umfasst den Bereich der europäischen Integration.
2. Die zweite Community verweist auf Integration als gesellschaftliche Aufgabe und Zukunftsfrage.
3. Die dritte Community bezieht sich auf die deutsche Geschichte, ist aber zunächst nicht weiter inhaltlich bestimmt. Hier könnte noch eine detailliertere Analyse vorgenommen werden.
4. Die vierte Community ist ein Artefakt, das sich aus den in allen Reden enthaltenen Anreden ergibt. Dies könnte ein Hinweis darauf sein, die Anreden vor der Analyse aus den Texten zu entfernen.
5. Die fünfte Community bestimmt Integration in Hinblick auf politische Investitionen in Bildung und Kultur.

Dies gibt einen ersten Eindruck davon, in welchen Kontexten Integration in politischen Reden der aktuellen deutschen Bundesregierung thematisiert wird.

## Literatur

Diesner, Jana und Kathleen M. Carley (2005): „Revealing social structure from texts: meta-matrix text analysis as a novel method for network text analysis“, Causal mapping for information systems and technology research: Approaches, advances, and illustrations , S. 81–108, http://andrew.cmu.edu/user/jdiesner/publications/DiesnerCarley_meta_matrix_text_analysis.pdf (zugegriffen am 8.9.2013).

Emirbayer, Mustafa (1997): „Manifesto for a relational sociology“, American Journal of Sociology 103/2, S. 281–317, http://www.jstor.org/stable/10.1086/231209 (zugegriffen am 29.10.2013).

Wittgenstein, Ludwig (2008): Philosophische Untersuchungen, hg. von. Joachim Schulte, Bibliothek Suhrkamp 1372, Frankfurt am Main: Suhrkamp.